# Merging delays with the schedules
With the daily schedule formatted into two tables in the first notebook, and delay information summarised into a single object in the second notebook, it's now time to merge the information so that we can easily get insights into how the trains ran that day.

## Full schedule with delays
We want today's timetable, but with the delays on a per stop basis.
Import the dataset formatted from the first notebook.

In [1]:
import pandas as pd
pd.options.display.max_rows = 10
df_stop_times = pd.read_pickle('stop_times.pickle')
df_stop_times

,trip_id,arrival_time,departure_time,stop_id
10,1--A.1260.122.48.M.8.55188157,03:52:00,03:52:00,2144243
11,1--A.1260.122.48.M.8.55188157,03:54:12,03:55:00,2141313
12,1--A.1260.122.48.M.8.55188157,03:57:30,03:57:30,214063
13,1--A.1260.122.48.M.8.55188157,03:58:42,03:58:42,214074
14,1--A.1260.122.48.M.8.55188157,04:01:24,04:01:24,2135234
...,...,...,...,...
1032254,WN18.1260.122.56.N.2.55187511,22:23:00,22:23:00,279536
1032255,WN18.1260.122.56.N.2.55187511,22:49:00,22:49:00,27874
1032256,WN18.1260.122.56.N.2.55187511,23:04:30,23:04:30,2790154
1032257,WN18.1260.122.56.N.2.55187511,23:12:00,23:12:00,284515


Add columns for arrival and departure times and delays, and schedule relationship. Mark all as N/A to start with, so we know which things never had real time updates

In [2]:
df_stop_times.insert(2, 'arrival_delay', 'N/A')
df_stop_times.insert(3, 'actual_arrival_time', 'N/A')
df_stop_times.insert(5, 'departure_delay', 'N/A')
df_stop_times.insert(6, 'actual_departure_time', 'N/A')
df_stop_times.insert(7, 'schedule_relationship', 'N/A')
df_stop_times

,trip_id,arrival_time,arrival_delay,actual_arrival_time,departure_time,departure_delay,actual_departure_time,schedule_relationship,stop_id
10,1--A.1260.122.48.M.8.55188157,03:52:00,N/A,N/A,03:52:00,N/A,N/A,N/A,2144243
11,1--A.1260.122.48.M.8.55188157,03:54:12,N/A,N/A,03:55:00,N/A,N/A,N/A,2141313
12,1--A.1260.122.48.M.8.55188157,03:57:30,N/A,N/A,03:57:30,N/A,N/A,N/A,214063
13,1--A.1260.122.48.M.8.55188157,03:58:42,N/A,N/A,03:58:42,N/A,N/A,N/A,214074
14,1--A.1260.122.48.M.8.55188157,04:01:24,N/A,N/A,04:01:24,N/A,N/A,N/A,2135234
...,...,...,...,...,...,...,...,...,...
1032254,WN18.1260.122.56.N.2.55187511,22:23:00,N/A,N/A,22:23:00,N/A,N/A,N/A,279536
1032255,WN18.1260.122.56.N.2.55187511,22:49:00,N/A,N/A,22:49:00,N/A,N/A,N/A,27874
1032256,WN18.1260.122.56.N.2.55187511,23:04:30,N/A,N/A,23:04:30,N/A,N/A,N/A,2790154
1032257,WN18.1260.122.56.N.2.55187511,23:12:00,N/A,N/A,23:12:00,N/A,N/A,N/A,284515


Then import the single delays object, and merge the data. Be warned, this is a very time consuming operation.

In [3]:
import pickle
import sys
sys.path.append('../')
from src.features.trip_objects import *
from src.features.trip_helper import *

merged_delays = pickle.load(open('merged_delays.pickle', 'rb'))

# iterate through all the trips we 
for trip in merged_delays.values():
    df_stop_times_this_trip = df_stop_times[(df_stop_times['trip_id'] == trip.trip_id)]
    for stop_time_update in trip.stop_time_updates.values():
        # some of these values might be 24:00, 25:00 etc to signify next day

        idx = df_stop_times_this_trip[(df_stop_times_this_trip['stop_id'] == stop_time_update.stop_id)].index
        if idx.empty:
            # it shouldn't be
            continue
            
        idx = idx.item()

        # calculate the real time
        actual_arrival_time = update_time('20190123', df_stop_times_this_trip.at[idx, 'arrival_time'],
                                               stop_time_update.arrival_delay)
        actual_departure_time = update_time('20190123', df_stop_times_this_trip.at[idx, 'departure_time'],
                                                 stop_time_update.departure_delay)

        # add the new values to the new columns
        df_stop_times.at[idx, 'arrival_delay'] = stop_time_update.arrival_delay
        df_stop_times.at[idx, 'actual_arrival_time'] = actual_arrival_time
        df_stop_times.at[idx, 'departure_delay'] = stop_time_update.departure_delay
        df_stop_times.at[idx, 'actual_departure_time'] = actual_departure_time
        df_stop_times.at[idx, 'schedule_relationship'] = stop_time_update.schedule_relationship

df_stop_times

,trip_id,arrival_time,arrival_delay,actual_arrival_time,departure_time,departure_delay,actual_departure_time,schedule_relationship,stop_id
10,1--A.1260.122.48.M.8.55188157,03:52:00,N/A,N/A,03:52:00,N/A,N/A,N/A,2144243
11,1--A.1260.122.48.M.8.55188157,03:54:12,N/A,N/A,03:55:00,N/A,N/A,N/A,2141313
12,1--A.1260.122.48.M.8.55188157,03:57:30,N/A,N/A,03:57:30,N/A,N/A,N/A,214063
13,1--A.1260.122.48.M.8.55188157,03:58:42,N/A,N/A,03:58:42,N/A,N/A,N/A,214074
14,1--A.1260.122.48.M.8.55188157,04:01:24,N/A,N/A,04:01:24,N/A,N/A,N/A,2135234
...,...,...,...,...,...,...,...,...,...
1032254,WN18.1260.122.56.N.2.55187511,22:23:00,N/A,N/A,22:23:00,N/A,N/A,N/A,279536
1032255,WN18.1260.122.56.N.2.55187511,22:49:00,N/A,N/A,22:49:00,N/A,N/A,N/A,27874
1032256,WN18.1260.122.56.N.2.55187511,23:04:30,N/A,N/A,23:04:30,N/A,N/A,N/A,2790154
1032257,WN18.1260.122.56.N.2.55187511,23:12:00,N/A,N/A,23:12:00,N/A,N/A,N/A,284515


Let's see if we have any arrival delay statistics:

In [4]:
df_stop_times[df_stop_times['arrival_delay'] != 'N/A']

,trip_id,arrival_time,arrival_delay,actual_arrival_time,departure_time,departure_delay,actual_departure_time,schedule_relationship,stop_id
18,1--A.1260.122.48.M.8.55188157,04:12:06,0,04:12:06,04:13:00,0,04:13:00,0,2015133
19,1--A.1260.122.48.M.8.55188157,04:15:30,0,04:15:30,04:17:00,0,04:17:00,0,2000336
143,1--B.1260.122.48.M.8.55188160,04:15:31,0,04:15:31,04:17:01,0,04:17:01,0,2000336
144,1--B.1260.122.48.M.8.55188160,04:19:48,0,04:19:48,04:20:48,0,04:20:48,0,2000396
145,1--B.1260.122.48.M.8.55188160,04:22:48,0,04:22:48,04:23:30,0,04:23:30,0,2000406
...,...,...,...,...,...,...,...,...,...
1031537,WN17.1260.122.56.N.2.55187512,19:05:06,0,19:05:06,19:05:36,378,19:11:54,0,2777192
1031546,WN17.1260.122.56.N.2.55187512,19:37:30,1734,20:06:24,19:38:30,1704,20:06:54,0,2780201
1031549,WN17.1260.122.56.N.2.55187512,19:57:30,1667,20:25:17,19:58:00,1854,20:28:54,0,278652
1031552,WN17.1260.122.56.N.2.55187512,20:23:00,1940,20:55:20,20:24:00,1910,20:55:50,0,2790142


In [5]:
df_stop_times.to_pickle('stop_times_merged.pickle')

## Trips
Now import the trips table, and merge the insights from the delay object into it.

In [6]:
df_trips = pd.read_pickle('trips.pickle')
df_trips

,route_id,service_id,trip_id
1,BNK_2a,1260.122.48,1--A.1260.122.48.M.8.55188157
19,APS_1a,1260.122.48,1--B.1260.122.48.M.8.55188160
37,APS_2a,1260.122.48,1--C.1260.122.48.M.8.55188159
55,APS_1a,1260.122.48,1--D.1260.122.48.M.8.55188306
73,APS_2a,1260.122.48,1--E.1260.122.48.M.8.55188307
...,...,...,...
57206,BMT_2,1260.122.16,W926.1260.122.16.V.4.55529028
57222,BMT_1,1260.122.56,WN11.1260.122.56.N.2.55190142
57232,BMT_2,1260.122.56,WN12.1260.122.56.N.2.55188260
57252,BMT_1,1260.122.56,WN17.1260.122.56.N.2.55187512


Add columns for the start and end timestamps, the delay averages and maxima, and the schedule relationship of each trip.

In [7]:
df_trips.insert(0, 'start_timestamp', 'N/A')
df_trips.insert(1, 'end_timestamp', 'N/A')
df_trips.insert(5, 'maximum_arrival_delay', 0)
df_trips.insert(6, 'average_arrival_delay', 0)
df_trips.insert(7, 'maximum_departure_delay', 0)
df_trips.insert(8, 'average_departure_delay', 0)
df_trips.insert(9, 'schedule_relationship', 0)
df_trips

,start_timestamp,end_timestamp,route_id,service_id,trip_id,maximum_arrival_delay,average_arrival_delay,maximum_departure_delay,average_departure_delay,schedule_relationship
1,N/A,N/A,BNK_2a,1260.122.48,1--A.1260.122.48.M.8.55188157,0,0,0,0,0
19,N/A,N/A,APS_1a,1260.122.48,1--B.1260.122.48.M.8.55188160,0,0,0,0,0
37,N/A,N/A,APS_2a,1260.122.48,1--C.1260.122.48.M.8.55188159,0,0,0,0,0
55,N/A,N/A,APS_1a,1260.122.48,1--D.1260.122.48.M.8.55188306,0,0,0,0,0
73,N/A,N/A,APS_2a,1260.122.48,1--E.1260.122.48.M.8.55188307,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
57206,N/A,N/A,BMT_2,1260.122.16,W926.1260.122.16.V.4.55529028,0,0,0,0,0
57222,N/A,N/A,BMT_1,1260.122.56,WN11.1260.122.56.N.2.55190142,0,0,0,0,0
57232,N/A,N/A,BMT_2,1260.122.56,WN12.1260.122.56.N.2.55188260,0,0,0,0,0
57252,N/A,N/A,BMT_1,1260.122.56,WN17.1260.122.56.N.2.55187512,0,0,0,0,0


Add the insights for each trip to the corresponding row of the table.

In [8]:
for trip in merged_delays.values():

    idx = df_trips[(df_trips['trip_id'] == trip.trip_id)].index
    if idx.empty:
        # it shouldn't be
        continue

    idx = idx.item()

    df_trips.at[idx, 'maximum_arrival_delay'] = trip.maximum_arrival_delay()
    df_trips.at[idx, 'average_arrival_delay'] = trip.average_arrival_delay()
    df_trips.at[idx, 'maximum_departure_delay'] = trip.maximum_departure_delay()
    df_trips.at[idx, 'average_departure_delay'] = trip.average_departure_delay()
    df_trips.at[idx, 'schedule_relationship'] = trip.overall_schedule_relationship()

df_trips

,start_timestamp,end_timestamp,route_id,service_id,trip_id,maximum_arrival_delay,average_arrival_delay,maximum_departure_delay,average_departure_delay,schedule_relationship
1,N/A,N/A,BNK_2a,1260.122.48,1--A.1260.122.48.M.8.55188157,0,0,0,0,0
19,N/A,N/A,APS_1a,1260.122.48,1--B.1260.122.48.M.8.55188160,25,1,0,0,0
37,N/A,N/A,APS_2a,1260.122.48,1--C.1260.122.48.M.8.55188159,9,4,0,0,0
55,N/A,N/A,APS_1a,1260.122.48,1--D.1260.122.48.M.8.55188306,1,0,0,0,0
73,N/A,N/A,APS_2a,1260.122.48,1--E.1260.122.48.M.8.55188307,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
57206,N/A,N/A,BMT_2,1260.122.16,W926.1260.122.16.V.4.55529028,0,0,0,0,0
57222,N/A,N/A,BMT_1,1260.122.56,WN11.1260.122.56.N.2.55190142,0,0,0,0,0
57232,N/A,N/A,BMT_2,1260.122.56,WN12.1260.122.56.N.2.55188260,332,47,92,13,0
57252,N/A,N/A,BMT_1,1260.122.56,WN17.1260.122.56.N.2.55187512,1940,805,1910,687,0


For every trip in the table, add the full start and end timestamps. This makes it easier to deal with information when a trip runs from very late to very early the next day, and the dates change.

In [9]:
for i in df_trips.index:
    departure_series = df_stop_times[df_stop_times['trip_id'] == df_trips.at[i, 'trip_id']]['departure_time']
    if len(departure_series) < 2:
        continue
    df_trips.at[i, 'start_timestamp'] = convert_to_timestamp('20190123', departure_series.iloc[0])
    df_trips.at[i, 'end_timestamp'] = convert_to_timestamp('20190123', departure_series.iloc[-1])

df_trips

,start_timestamp,end_timestamp,route_id,service_id,trip_id,maximum_arrival_delay,average_arrival_delay,maximum_departure_delay,average_departure_delay,schedule_relationship
1,2019-01-23 03:52:00,2019-01-23 04:17:00,BNK_2a,1260.122.48,1--A.1260.122.48.M.8.55188157,0,0,0,0,0
19,2019-01-23 04:17:01,2019-01-23 04:58:00,APS_1a,1260.122.48,1--B.1260.122.48.M.8.55188160,25,1,0,0,0
37,2019-01-23 04:58:01,2019-01-23 05:17:00,APS_2a,1260.122.48,1--C.1260.122.48.M.8.55188159,9,4,0,0,0
55,2019-01-23 05:17:01,2019-01-23 05:58:00,APS_1a,1260.122.48,1--D.1260.122.48.M.8.55188306,1,0,0,0,0
73,2019-01-23 05:58:01,2019-01-23 06:35:00,APS_2a,1260.122.48,1--E.1260.122.48.M.8.55188307,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
57206,2019-01-23 13:06:01,2019-01-23 16:35:00,BMT_2,1260.122.16,W926.1260.122.16.V.4.55529028,0,0,0,0,0
57222,2019-01-23 03:42:00,2019-01-23 05:46:00,BMT_1,1260.122.56,WN11.1260.122.56.N.2.55190142,0,0,0,0,0
57232,2019-01-23 05:46:01,2019-01-23 11:02:00,BMT_2,1260.122.56,WN12.1260.122.56.N.2.55188260,332,47,92,13,0
57252,2019-01-23 17:47:01,2019-01-23 22:17:00,BMT_1,1260.122.56,WN17.1260.122.56.N.2.55187512,1940,805,1910,687,0


What were the worst delays?

In [10]:
df_trips.sort_values(by=['maximum_departure_delay'], ascending=False).head(n=10)

,start_timestamp,end_timestamp,route_id,service_id,trip_id,maximum_arrival_delay,average_arrival_delay,maximum_departure_delay,average_departure_delay,schedule_relationship
56746,2019-01-23 18:33:01,2019-01-23 21:30:00,BMT_1,1260.122.48,W581.1260.122.48.V.4.55187453,5056,2214,5056,2358,0
56457,2019-01-23 15:01:00,2019-01-23 18:02:00,BMT_2,1260.122.48,W560.1260.122.48.V.4.55186785,3894,2985,3894,2964,0
56852,2019-01-23 21:18:01,2019-01-23 23:40:48,BMT_1,1260.122.48,W591.1260.122.48.V.4.55187824,3951,559,2508,323,0
52749,2019-01-23 14:15:01,2019-01-23 17:24:00,CCN_1a,1260.122.60,N155.1260.122.60.V.8.55187399,2512,2159,2482,2103,0
56689,2019-01-23 17:31:19,2019-01-23 19:54:30,BMT_1,1260.122.124,W575.1260.122.124.V.8.55191035,1706,623,2174,702,0
57252,2019-01-23 17:47:01,2019-01-23 22:17:00,BMT_1,1260.122.56,WN17.1260.122.56.N.2.55187512,1940,805,1910,687,0
56592,2019-01-23 16:47:01,2019-01-23 19:03:00,BMT_1,1260.122.60,W569.1260.122.60.V.8.55188354,1342,406,1802,450,0
398,2019-01-23 13:57:01,2019-01-23 15:30:00,APS_1a,1260.122.120,10-K.1260.122.120.M.8.55186888,1742,155,1742,69,1
56532,2019-01-23 16:17:01,2019-01-23 19:06:00,BMT_1,1260.122.124,W565.1260.122.124.V.8.55189034,1383,634,1474,637,0
56436,2019-01-23 15:18:01,2019-01-23 18:25:00,BMT_1,1260.122.16,W559.1260.122.16.V.8.55189171,1354,633,1423,673,0


84 minutes!

In [11]:
df_trips.to_pickle('trips_insights.pickle')